In [1]:
# Written by Chunyu
# This file is for testing /data/seanchoi/airflow/db_utils/push_sec_reports.py and /data/seanchoi/airflow/db_utils/search_db.py

import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  # .ipynb file
sys.path.append(project_root)

from collections import Counter
from datetime import datetime
from bson import ObjectId
from datetime import datetime
from db_utils import *

In [2]:
company_name = "Amazon.com, Inc."
results_ticker = list(filter_files(COMPANY_TICKER_DATA, {"Company Name": company_name}))
results_ticker

# NVIDIA Corporation

[{'_id': ObjectId('6765822dc2568cb07c25163e'),
  'Ticker': 'AMZN',
  'Company Name': 'Amazon.com, Inc.',
  'Industry': 'Internet Retail',
  'Market Cap': '2,424.08B',
  'metadata': {'uploaded_date': '2024-12-20'}}]

In [3]:
company_ticker_id = ObjectId("6765822dc2568cb07c25163e")  # 示例 company_ticker_id
start = datetime(2023, 1, 1)
end = datetime(2025, 12, 31)

quarter_reports = find_quarter_reports(
    company_ticker_id=company_ticker_id,
    start_date=start,
    end_date=end
)

annual_reports = find_annual_reports(
    company_ticker_id=company_ticker_id,
    start_date=start,
    end_date=end
)

transcripts = find_transcripts(
    company_ticker_id=company_ticker_id,
    start_date=start,
    end_date=end
)

In [5]:
len(quarter_reports), len(annual_reports), len(transcripts)

(7, 2, 5)

In [6]:
final_response = query_across_collections("Amazon.com, Inc.", "2022-01-01", "2026-01-01")
# ## to test news
# final_response = query_across_collections("Motors Liquidation Co", "2002-01-01", "2023-01-01")
# print(final_response)

ticker_id:  6765822dc2568cb07c25163e


In [ ]:
def analyze_final_response(final_response):
    print("========== 年报 (10-K) 信息 ==========")
    annual_reports = final_response.get("annual_reports", [])
    print(f"总数量：{len(annual_reports)}")

    # 年份分布
    annual_years = [r.get("year") for r in annual_reports if "year" in r]
    year_counter = Counter(annual_years)
    print("按年份分布（10-K）：")
    for year, count in sorted(year_counter.items()):
        print(f"  {year}: {count} 份")

    print("\n========== 季报 (10-Q) 信息 ==========")
    quarter_reports = final_response.get("quarter_reports", [])
    print(f"总数量：{len(quarter_reports)}")

    # 年份分布
    quarter_years = [r.get("year") for r in quarter_reports if "year" in r]
    quarter_counter = Counter(quarter_years)
    print("按年份分布（10-Q）：")
    for year, count in sorted(quarter_counter.items()):
        print(f"  {year}: {count} 份")

    # 季度分布
    quarters = [r.get("quarter", "缺失") for r in quarter_reports]
    quarter_freq = Counter(quarters)
    print("季度分布（10-Q）：")
    for q, count in quarter_freq.items():
        print(f"  {q.upper()}: {count} 份")
        
    print("\n========== 电话会议纪要 (Transcript) 信息 ==========")
    transcripts = final_response.get("transcripts", [])
    print(f"总数量：{len(transcripts)}")

    # 年份分布
    transcript_years = [r.get("year") for r in transcripts if "year" in r]
    transcript_year_counter = Counter(transcript_years)
    print("按年份分布（Transcript）：")
    for year, count in sorted(transcript_year_counter.items()):
        print(f"  {year}: {count} 份")

    # 季度分布
    transcript_quarters = [r.get("quarter", "缺失") for r in transcripts]
    transcript_quarter_counter = Counter(transcript_quarters)
    print("季度分布（Transcript）：")
    for q, count in transcript_quarter_counter.items():
        print(f"  {q.upper()}: {count} 份")

# 调用分析函数
analyze_final_response(final_response)


========== 年报 (10-K) 信息 ==========
总数量：2
按年份分布（10-K）：
  2024: 1 份
  2025: 1 份

========== 季报 (10-Q) 信息 ==========
总数量：7
按年份分布（10-Q）：
  2023: 3 份
  2024: 3 份
  2025: 1 份
季度分布（10-Q）：
  Q2: 3 份
  Q3: 2 份
  Q4: 2 份

========== 电话会议纪要 (Transcript) 信息 ==========
总数量：5
按年份分布（Transcript）：
  2024: 3 份
  2025: 2 份
季度分布（Transcript）：
  Q2: 2 份
  Q3: 1 份
  Q4: 1 份
  Q1: 1 份
会议类型分布：
  未知: 5 份


In [ ]:
# save sec_reports to json file for better preview
import bson.json_util
import json, bson

save_file = "./final_response.json"
with open(save_file, "w", encoding="utf-8") as f:
    json.dump(final_response, f, default=bson.json_util.default, ensure_ascii=False, indent=2)

[{'_id': ObjectId('6765822dc2568cb07c25163e'),
  'Ticker': 'AMZN',
  'Company Name': 'Amazon.com, Inc.',
  'Industry': 'Internet Retail',
  'Market Cap': '2,424.08B',
  'metadata': {'uploaded_date': '2024-12-20'}}]

In [ ]:
# save sec_reports to json file for better preview
from db_utils import get_db, SEC_REPORT_COLLECTION, save_json

_, db = get_db()
collection = db[SEC_REPORT_COLLECTION]
cursor = collection.find().sort("_id", -1).limit(10)

save_file = "./sec_reports_sorted.json"
save_json(cursor, save_file)